# Intro to PyTorch and Neural Network Training
#### *Author* : Johnathon Li (johnathon.li@momenta.ai)

In [ ]:
# import necessary libraries
import torch
import numpy as np

print(f"Using torch version {torch.__version__}")

In [ ]:
import platform

pl = platform.system().lower()
if "darwin" in pl:
    # M1 GPU acceleration not stable as of yet so proceed at your own risk
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
else:
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Using device {device}")

In [ ]:
# Function for setting the seed
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)

## Tensors in PyTorch

Tensors are generalizations of scalars, vectors, matrices and so on. You can see a scalar as a 0-th dimensional tensor, a vector 1D, a matrix 2D, etc. Tensors are similar to NumPy arrays, except they are **optimized for automatic differentiation and can run on GPUs**.

In [ ]:
# create a tensor
np.set_printoptions(precision=3)
a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=int)
t_a = torch.tensor(a) # make a tensor
t_b = torch.from_numpy(b) # convert numpy array to tensor

In [ ]:
print(t_a)
print(t_b)

In [ ]:
t_ones = torch.ones(2, 3) # make a matrix of dim 2 x 3 filled with value 1
print(t_ones)
t_ones.shape

In [ ]:
# generate a tensor of random values
t_rand = torch.rand(2,3)
print(t_rand)

### Manipulating Datatype and Shape of Tensors

The function `torch.to()` can be used to change data type of a tensor to a desired type.

In [ ]:
t_a_new = t_a.to(torch.float)
print(f"t_a type = {t_a.dtype}")
print(f"t_a_new type = {t_a_new.dtype}")

Some operations require that input tensors have a certain dimension (rank) associated with a certain number of elements. We can change the shape of them with the functions `torch.transpose()`, `torch.reshape()` (or `torch.view()`), and `torch.squeeze()`.

In [ ]:
# transposing a tensor
t = torch.rand(3, 5)
t_T = t.transpose(0, 1) # swap dim 0 and 1
print(t.shape, " --> ", t_T.shape)

In [ ]:
# reshaping a tensor
t = torch.zeros(30)
t_reshape_1 = t.reshape(3, 10)
t_reshape_2 = t.view(3,10) # the same
print(t.shape, " --> ", t_reshape_1.shape)

In [ ]:
# removing unnecessary dimensions (those dimensions of value 1 are not needed)
t = torch.zeros(1, 2, 1, 4, 1)
t_sqz = t.squeeze(2) # remove 3rd dim of 1
print(t.shape, " --> ", t_sqz.shape)

### Mathematical Operations on Tensors

In [ ]:
t1 = 2*torch.rand(5,2) - 1
t2 = torch.normal(mean=0, std=1, size=(5,2))
print(t1)
print(t2)

##### Element Wise Matrix Multiplication

In [ ]:
t3 = t1.multiply(t2)
print(t3)

##### Matrix-Matrix Product

In [ ]:
t4 = torch.matmul(t1, t2.transpose(0, 1))
print(t1.shape, " x ", t2.transpose(0,1).shape, " = ", t4.shape)

##### Sum, Mean, Standard Deviation, and Norm

In [ ]:
t_mean = t1.mean() # across all rows and columns
t_mean_row = t1.mean(axis=1)
t_mean_col = t1.mean(axis=0)
print(t_mean)
print(t_mean_row)
print(t_mean_col)

In [ ]:
t_sum = t1.sum()
t_std = t1.std()
print(f"sum = {t_sum}")
print(f"standard deviation = {t_std}")

In [ ]:
t_norm = torch.linalg.norm(t1, ord=2, dim=1) # computer L2 norm of tensor for each roch
print(t_norm)

### Split, Stack and Concatenate Tensors

In [ ]:
# divide a tensor into "n" equal chunks
t = torch.rand(6)
print(f"t = {t}")
t_splits = torch.chunk(t, chunks=3)
[f"Chunk {i} = {ch}" for i, ch in enumerate(t_splits)]

In [ ]:
# stack or concatenate tensors
A = torch.ones(1, 3)
B = torch.zeros(4, 3)
C = torch.cat([A, B], axis=0)
print(f"A = {A}")
print(f"B = {B}")
print(f"C = {C}")

## Training Neural Networks with PyTorch

In the next section, we will build a simple neural network to classify handwritten digits from the MNIST data set.

In [ ]:
import sklearn
import torchvision

%matplotlib inline 
import matplotlib.pyplot as plt

### Dataset Loading and Processing

In [ ]:
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_openml
from tqdm.notebook import tqdm

dataset = fetch_openml("mnist_784", version=1)

In [ ]:
# see the contents in the dataset
print(list(dataset))
print(dataset.data.shape) # the images
print(dataset.target.shape) # the labels

Let's visualize the classes in the MNIST dataset.

In [ ]:
# Visualize some dataset instances
# plot some digits
X, y = dataset.data.values, dataset.target.astype(int).values

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten() # 1D iterator over array
for i in range(10):
    img = X[y==i][0].reshape(28,28)
    ax[i].imshow(img, cmap="Greys")
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

It is important to preprocess the data in some way before input into the neural network. They are sensitive to features with large magnitudes.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scale = MinMaxScaler(feature_range=(-1, 1))
X_processed = min_max_scale.fit_transform(X)

Let's convert the `sklearn` dataset into one streamlined for `torch`.

When model training is running, we will train on train data and evaluate during training on validation data. We will additionally reserve a subset of the data as test data for model evaluation after training is complete.

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils import data

def to_torch(x):
    if torch.is_tensor(x):
        return x
    return torch.from_numpy(x).to(torch.float32)

When training any Machine Learning algorithm, it is important to have representative instances of the general dataset population. A model will have poor performance if the training instance ratios it sees are not similar to the ratios of the real dataset. This type of equal sampling is called *stratified sampling.*

In [ ]:
test_ratio = 0.15
val_ratio  = 0.2
# split into temp and test
X_temp, X_test, y_temp, y_test = train_test_split(X_processed, y, test_size=test_ratio, stratify=y)
# split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_ratio, stratify=y_temp)

# sklearn -> torch
X_train_t, y_train_t = to_torch(X_train), to_torch(y_train)
X_test_t, y_test_t  = to_torch(X_test), to_torch(y_test)
X_val_t, y_val_t = to_torch(X_val), to_torch(y_val)

train_dataset = data.TensorDataset(X_train_t, y_train_t)
test_dataset  = data.TensorDataset(X_test_t, y_test_t)
val_dataset   = data.TensorDataset(X_val_t, y_val_t)

In PyTorch, the `data.DataLoader` class handles feeding data to the model during training and evaluation.

In [ ]:
b_size = 100

train_loader = data.DataLoader(train_dataset, batch_size=b_size, shuffle=True, drop_last=True, pin_memory=True)
val_loader  = data.DataLoader(val_dataset, batch_size=b_size, shuffle=False, drop_last=False)
test_loader = data.DataLoader(test_dataset, batch_size=b_size, shuffle=False, drop_last=False)

### Implementing a Neural Network in PyTorch

We will implement a simple Multilayer Perceptron to perform a logistic regression task.

<img src="mlp.png" alt="mlp" width="600"/>

The foundation of building neural networks from PyTorch is done through `torch.nn.Module`. These are the building blocks.

In [ ]:
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, num_features=784, num_classes=10, hidden_sizes=[50]):
        """ 
        To add more layers in your network, add additional layer sizes to the "hidden_sizes" hyperparam
        """
        super().__init__()
        layers = []
        layer_sizes = [num_features] + hidden_sizes + [num_classes]
        for i in range(1, len(layer_sizes)):
            layers.append(nn.Linear(layer_sizes[i-1], layer_sizes[i]))
        layers.append(nn.Sigmoid()) # added for logistic regression task
    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

### Activation Functions
Since we are doing a Logistic Regression, the activation function of choice is a sigmoid. There are a myriad of activation functions for deep neural networks, each with its pros and cons. See a summary below or visit this [link](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions).

<img src="act_fns.png" width="450">

In [ ]:
log_reg = LogisticRegression()
print(log_reg)

Neural Network weight initialization is very important as successful training is dependent consistent gradient and variance flow across layers. For more details, see [here](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.calculate_gain).

In [ ]:
def init_weights(model, factor=1):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            val = np.random.normal(loc=0.0, scale=0.1)
            param.data.fill_(val)
        else:
            nn.init.xavier_uniform_(param)

init_weights(log_reg)

We will now define the loss function and optimizers for our model. Since we are performing a logistic regression, a suitable loss function is Cross Entropy. For more details on loss functions, see [this link](https://towardsdatascience.com/common-loss-functions-in-machine-learning-46af0ffc4d23).

The optimization controls how fast our weights change (i.e. how big of a step is taken when performing gradient descent).

In [ ]:
loss_module = nn.CrossEntropyLoss() # compute softmax then cross entropy
optim = torch.optim.SGD(log_reg.parameters(), lr=0.1)

In [ ]:
def train(model, optimizer, train_loader, val_loader, loss_module, num_epochs=50, verbose=False):
    # set the model to train mode
    model.to(device)

    train_losses = []
    train_accuracies = []
    val_accuracies = []

    pbar = tqdm(range(num_epochs))

    for epoch in pbar:
        train_loss, train_acc, val_acc = 0.0, 0, 0
        train_instances = 0
        val_instances = 0

        model.train()
        for features, labels in train_loader:
            # move tensors to GPU (or CPU if no GPU)
            features = features.to(device)
            labels = labels.to(device)

            predictions = model(features).squeeze(1)
            loss = loss_module(predictions, labels.long())
            
            # MUST ALWAYE DO THIS STEP
            optimizer.zero_grad() # clear out residual gradients (very important)

            loss.backward() # back propogation
            optimizer.step() # run gradient descent

            # metrics
            train_loss += loss.item()
            train_acc += (torch.argmax(predictions, dim=1) == labels).sum().item()

            train_instances += features.shape[0]

        train_losses.append(train_loss/train_instances)
        train_accuracies.append(train_acc/train_instances)

        # run validation loop
        model.eval()
        with torch.no_grad():
            for features, labels in val_loader:
                # same as train but only forward prop
                features = features.to(device)
                labels = labels.to(device)

                predictions = model(features).squeeze(1)
                val_acc += (torch.argmax(predictions, dim=1) == labels).sum().item()

                val_instances += features.shape[0]

        val_accuracies.append(val_acc/val_instances)

        if verbose:
            pbar.set_description(f"Epoch = {epoch}/{num_epochs} | "
                    f"Train MSE: {train_loss/train_instances:.4f} | " 
                    f"Train Acc: {train_acc/train_instances*100:.4f}% | " 
                    f"Val Acc: {val_acc/val_instances*100:.4f}%")

    return (train_losses, train_accuracies, val_accuracies)


In [ ]:
num_epochs = 50
train_losses, train_accuracies, val_accuracies = train(log_reg, optim, train_loader, val_loader, 
                                                    loss_module, num_epochs=num_epochs, verbose=True)

### Evaluation of the Neural Network

In [ ]:
# plot the loss and accuracies

plt.figure(figsize=(12,6))
idx = [i for i in range(num_epochs)]
plt.plot(idx, train_losses, label="Loss")
plt.title("Cross Entropy Loss vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.legend(loc="lower left")
plt.plot()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(idx, train_accuracies, label="Training Acc")
plt.plot(idx, val_accuracies, label="Validation Acc")
plt.title("Accuracies vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid()
plt.legend(loc="lower right")
plt.plot()

You should see that after the "elbow" of the training graphs, the validation and training accuracies diverge from each other. This gap betweent the two curves is indicative of overfitting. Some things you can do are lower the learning rate, decrease the number of hidden layers, lower the number of epochs, etc.

Let's now evaluate the model on the test set.

In [ ]:
@torch.no_grad()
def evaluate(model, test_loader):
    model.to(device)
    model.eval()
    num_examples = 0
    acc = 0.0
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        predictions = model(features)
        num_examples += features.shape[0]
        acc += (torch.argmax(predictions.cpu(), axis=1) == labels.cpu()).sum().item()
    acc /= num_examples
    print(f"Test Accuracy = {acc*100:.4f}%.")

In [ ]:
evaluate(log_reg, test_loader)

The test accuracy is similar to the train and validation accuracies. Play around with the model to try and improve your score!

Some things you can try are:
1. Data preprocessing - transform the training inputs more to lower the bias of the model.
2. Hyperparameter tuning - change the number of layers, the hidden dimension size, the learning rate, etc.

### Saving and Reloading Model

Saving the full model

In [ ]:
model_path = "simple_log_reg.pt"
torch.save(log_reg, model_path)

In [ ]:
# reload the model
log_reg_reload = torch.load(model_path)
# verify that the model is the same
print(log_reg_reload.eval())
# now verfiy the weights are the same by evaluating it on test set
evaluate(log_reg_reload, test_loader)

Saving only the learned parameters

In [ ]:
path = "simple_log_reg_state.pt"
torch.save(log_reg.state_dict(), path)

# reload saved parameters to reconstruct model
log_reg_reload = LogisticRegression()
log_reg_reload.load_state_dict(torch.load(path)) 

# now verfiy the weights are the same by evaluating it on test set
evaluate(log_reg_reload, test_loader)